<a href="https://colab.research.google.com/github/ngarciap/AnalisisExploratorioDeDatos/blob/main/An%C3%A1lisis_Bivariado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Análisis Bivariado con Dataset de Prescripciones del mercado**
***Análisis Bivariado / Asociado a mi pregunta SMART***

In [ ]:
# Importar librerias
import pandas as pd
import numpy as np
import os

# Matplotlib y  seaborn para gráficos
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

matplotlib.rcParams['font.size'] = 8
matplotlib.rcParams['figure.dpi'] = 200

#from google.colab import drive

""" Cargar Dataframes desde Google Drive
drive.mount('/content/drive', force_remount=True)
df_producto = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Analisis Exploratorio de Datos/Bases/Market COL Jun/CUP_PRODUTO.TXT', sep='	', encoding='latin1', dtype=str)
df_laboratorio = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Analisis Exploratorio de Datos/Bases/Market COL Jun/CUP_LABORATORIO.TXT', sep='	', encoding='latin1', dtype=str)
df_prescripciones = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Analisis Exploratorio de Datos/Bases/Market COL Jun/CUP_FATO.TXT', sep='	', encoding='latin1', dtype=str)
df_category = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Analisis Exploratorio de Datos/Bases/Market COL Jun/Category.xlsx', engine='openpyxl', dtype=str)
"""
# Cargar Dataframes desde máquina local
df_producto = pd.read_csv(os.path.join(os.getcwd() + "\Bases", "CUP_PRODUTO.TXT"), sep='	', encoding='latin1', dtype=str)
df_laboratorio = pd.read_csv(os.path.join(os.getcwd() + "\Bases", "CUP_LABORATORIO.TXT"), sep='	', encoding='latin1', dtype=str)
df_prescripciones = pd.read_csv(os.path.join(os.getcwd() + "\Bases", "CUP_FATO.TXT"), sep='	', encoding='latin1', dtype=str)
df_category = pd.read_excel(os.path.join(os.getcwd() + "\Bases", "Category.xlsx"), engine='openpyxl', dtype=str)

**Se procede a Cruzar la Tabla principal de Rx con Productos y Laboratorios para poder obtener un Dataframe que me inidque prescripciones por laboratorio de todo el Dataset**

In [ ]:
# Alistamiento de tabla de productos para traer el Laboratorio y cruzar con los articulos claisificados corporativamente
df_producto_completo = pd.merge(df_producto, df_laboratorio, how='inner', left_on='PRO_LAB_ID', right_on='LAB_ID')
df_producto_completo = pd.merge(df_producto_completo, df_category, how = 'inner', on='PRO_ID')

col_descriptivas = df_prescripciones.columns[:df_prescripciones.columns.get_loc('ESP_ID') + 1].tolist()
col_meses = df_prescripciones.columns[df_prescripciones.columns.get_loc('ESP_ID') + 1:].tolist()
df_prescripciones[col_meses] = df_prescripciones[col_meses].astype(int)

df_prescripciones = df_prescripciones.groupby(col_descriptivas)[col_meses].sum().reset_index()
df_prescripciones = pd.melt(df_prescripciones, id_vars=col_descriptivas, var_name='PERIODO', value_name='VALOR')
df_prescripciones = df_prescripciones[df_prescripciones['VALOR'] != 0]

# Cruce de Prescrupciones con Producto y filtrado de Columnas de interés
df_prescripciones = pd.merge(df_prescripciones, df_producto_completo, how='inner', on='PRO_ID')
df_prescripciones_agrupado = df_prescripciones.groupby(['LAB_DESC', 'PERIODO'])['VALOR'].sum().reset_index()

# Se filtra el último mes de la base para analizarlo
df_prescripciones_agrupado = df_prescripciones_agrupado[df_prescripciones_agrupado['PERIODO'] == 'MES59']